# Classificador ótimo de Bayes

## Probabilidade a priori

- <big>${P(c|x) = \frac{x}{c}}$</big>

## Probabilidade a posteriori


- <big>${P(c_j|X_t) = \frac{P(x_t|c_j).P(c_j)}{P(x_t)}}$</big>

In [1]:
import pandas as pd

In [2]:
hist = ['Ruim','Desconhecida','Desconhecida','Desconhecida',
        'Desconhecida','Desconhecida','Ruim','Ruim','Boa',
        'Boa','Boa','Boa','Boa','Ruim']
divida = ['Alta','Alta','Baixa','Baixa','Baixa','Baixa',
          'Baixa','Baixa','Baixa','Alta','Alta','Alta','Alta','Alta']
garantia = ['Nenhuma','Nenhuma','Nenhuma','Nenhuma','Nenhuma',
           'Adequada','Nenhuma', 'Adequada','Nenhuma', 'Adequada','Nenhuma',
           'Nenhuma','Nenhuma','Nenhuma']
renda = ['< 15.000', '>= 15.000 a 35.000', '>= 15.000 a 35.000',
        '< 15.000', '> 35.000', '> 35.000','< 15.000', '> 35.000',
         '> 35.000','> 35.000','< 15.000','>= 15.000 a 35.000',
         '> 35.000','>= 15.000 a 35.000']
risco = ['Alto','Alto','Moderado','Alto','Baixo','Baixo','Alto',
         'Moderado','Baixo','Baixo','Alto','Moderado','Baixo','Alto']

In [3]:
len(hist), len(divida), len(garantia), len(renda), len(risco)

(14, 14, 14, 14, 14)

In [4]:
data = {'Historico':hist, 'Divida':divida,'Garantias':garantia, 'Renda':renda, 'Risco':risco}

In [5]:
df = pd.DataFrame(data)
df

,Historico,Divida,Garantias,Renda,Risco
0,Ruim,Alta,Nenhuma,< 15.000,Alto
1,Desconhecida,Alta,Nenhuma,>= 15.000 a 35.000,Alto
2,Desconhecida,Baixa,Nenhuma,>= 15.000 a 35.000,Moderado
3,Desconhecida,Baixa,Nenhuma,< 15.000,Alto
4,Desconhecida,Baixa,Nenhuma,> 35.000,Baixo
5,Desconhecida,Baixa,Adequada,> 35.000,Baixo
6,Ruim,Baixa,Nenhuma,< 15.000,Alto
7,Ruim,Baixa,Adequada,> 35.000,Moderado
8,Boa,Baixa,Nenhuma,> 35.000,Baixo
9,Boa,Alta,Adequada,> 35.000,Baixo


In [6]:
def indep_event(target: list, space: list, event: int = 1) -> float:
    p = 1
    target = len(target)
    space = len(space)
    
    for i in range(event):
        p *= round((target / space), 2)
        
    return p

In [7]:
df['Risco'].unique()

array(['Alto', 'Moderado', 'Baixo'], dtype=object)

In [8]:
len(df)

14

In [9]:
# Probabilidade a priori das classes
p_class = [indep_event(df[df['Risco']==classe], df['Risco']) for classe in df['Risco'].unique()]
p_class

[0.43, 0.21, 0.36]

In [10]:
var_class = [len(df[col].unique()) for col in df.columns]
var_class

[3, 2, 2, 3, 3]

In [11]:
dct_class = {}
p = 1
for key, val in zip(df.columns, var_class):
    dct_class[key] = val
    
    if key != 'Risco':
        p *= val
    else:
        p *= val
        break
        
p, dct_class

(108, {'Historico': 3, 'Divida': 2, 'Garantias': 2, 'Renda': 3, 'Risco': 3})

In [12]:
df.columns[:-1]

Index(['Historico', 'Divida', 'Garantias', 'Renda'], dtype='object')

In [13]:
prob_cond = dct_class['Risco'] * (
    dct_class['Historico'] * dct_class['Divida'] * dct_class['Garantias'] * dct_class['Renda'])

In [14]:
prob_cond

108

In [15]:
prob_apriori = {key:val for key, val in zip(df['Risco'].unique(), p_class)}
prob_apriori

{'Alto': 0.43, 'Moderado': 0.21, 'Baixo': 0.36}

In [16]:
x = df.query('Historico == "Boa" & Divida == "Alta" & Garantias == "Nenhuma" & Renda == "> 35.000"')
x

,Historico,Divida,Garantias,Renda,Risco
12,Boa,Alta,Nenhuma,> 35.000,Baixo


In [17]:
# Probabilidade a posteriori
# Risco Alto
108 * prob_apriori['Alto'] / (len(x) / len(df))

650.16

In [18]:
# Risco Moderado
108 * prob_apriori['Moderado'] / (len(x) / len(df))

317.52000000000004

In [19]:
# Risco Baixo
108 * prob_apriori['Baixo'] / (len(x) / len(df))

544.3199999999999

## Naive Bayes

In [20]:
x

,Historico,Divida,Garantias,Renda,Risco
12,Boa,Alta,Nenhuma,> 35.000,Baixo


${P(A,x) = P(Risco A) . P(Risco A, X Hist) . P(Risco A, X Divida) . P(Risco A, X Garantias) . P(Risco A, X Renda)}$

<big>${P(A) = \frac{P(A,x)}{\frac{\sum{(P(A,x),P(M,x),P(B,x))}}{100}}}$</big>

---

${P(M,x) = P(Risco M) . P(Risco M, X Hist) . P(Risco M, X Divida) . P(Risco M, X Garantias) . P(Risco M, X Renda)}$

<big>${P(M) = \frac{P(M,x)}{\frac{\sum{(P(A,x),P(M,x),P(B,x))}}{100}}}$</big>

---

${P(B,x) = P(Risco B) . P(Risco B, X Hist) . P(Risco B, X Divida) . P(Risco B, X Garantias) . P(Risco B, X Renda)}$

<big>${P(B) = \frac{P(B,x)}{\frac{\sum{(P(A,x),P(M,x),P(B,x))}}{100}}}$</big>

In [21]:
hist_alto = df.query('Risco == "Alto"')
hist_alto

,Historico,Divida,Garantias,Renda,Risco
0,Ruim,Alta,Nenhuma,< 15.000,Alto
1,Desconhecida,Alta,Nenhuma,>= 15.000 a 35.000,Alto
3,Desconhecida,Baixa,Nenhuma,< 15.000,Alto
6,Ruim,Baixa,Nenhuma,< 15.000,Alto
10,Boa,Alta,Nenhuma,< 15.000,Alto
13,Ruim,Alta,Nenhuma,>= 15.000 a 35.000,Alto


In [22]:
hist_boa_alto = df.query('Historico == "Boa" & Risco == "Alto"')
hist_boa_alto

,Historico,Divida,Garantias,Renda,Risco
10,Boa,Alta,Nenhuma,< 15.000,Alto


In [23]:
hist_div_alto = df.query('Risco == "Alto" & Divida == "Alta"')
hist_div_alto

,Historico,Divida,Garantias,Renda,Risco
0,Ruim,Alta,Nenhuma,< 15.000,Alto
1,Desconhecida,Alta,Nenhuma,>= 15.000 a 35.000,Alto
10,Boa,Alta,Nenhuma,< 15.000,Alto
13,Ruim,Alta,Nenhuma,>= 15.000 a 35.000,Alto


In [24]:
hist_gar_alto = df.query('Risco == "Alto" & Garantias == "Nenhuma"')
hist_gar_alto

,Historico,Divida,Garantias,Renda,Risco
0,Ruim,Alta,Nenhuma,< 15.000,Alto
1,Desconhecida,Alta,Nenhuma,>= 15.000 a 35.000,Alto
3,Desconhecida,Baixa,Nenhuma,< 15.000,Alto
6,Ruim,Baixa,Nenhuma,< 15.000,Alto
10,Boa,Alta,Nenhuma,< 15.000,Alto
13,Ruim,Alta,Nenhuma,>= 15.000 a 35.000,Alto


In [25]:
hist_renda_alto = df.query('Risco == "Alto" & Renda == "> 35.000"')
hist_renda_alto

,Historico,Divida,Garantias,Renda,Risco


In [26]:
pa = (len(hist_alto) / len(df)) \
* (len(hist_boa_alto) / len(hist_alto)) \
* (len(hist_div_alto) / len(hist_alto)) \
* (len(hist_gar_alto) / len(hist_alto)) \
* ((len(hist_renda_alto)+1) / len(hist_alto)) 
pa

0.007936507936507936

In [27]:
hist_mod = df.query('Risco == "Moderado"')
hist_mod

,Historico,Divida,Garantias,Renda,Risco
2,Desconhecida,Baixa,Nenhuma,>= 15.000 a 35.000,Moderado
7,Ruim,Baixa,Adequada,> 35.000,Moderado
11,Boa,Alta,Nenhuma,>= 15.000 a 35.000,Moderado


In [28]:
hist_boa_mod = df.query('Historico == "Boa" & Risco == "Moderado"')
hist_boa_mod

,Historico,Divida,Garantias,Renda,Risco
11,Boa,Alta,Nenhuma,>= 15.000 a 35.000,Moderado


In [29]:
hist_div_mod = df.query('Divida == "Alta" & Risco == "Moderado"')
hist_div_mod

,Historico,Divida,Garantias,Renda,Risco
11,Boa,Alta,Nenhuma,>= 15.000 a 35.000,Moderado


In [30]:
hist_gar_mod = df.query('Garantias == "Nenhuma" & Risco == "Moderado"')
hist_gar_mod

,Historico,Divida,Garantias,Renda,Risco
2,Desconhecida,Baixa,Nenhuma,>= 15.000 a 35.000,Moderado
11,Boa,Alta,Nenhuma,>= 15.000 a 35.000,Moderado


In [31]:
hist_renda_mod = df.query('Renda == "> 35.000" & Risco == "Moderado"')
hist_renda_mod

,Historico,Divida,Garantias,Renda,Risco
7,Ruim,Baixa,Adequada,> 35.000,Moderado


In [32]:
len(hist_mod), len(df)

(3, 14)

In [33]:
len(hist_boa_mod), len(hist_mod)

(1, 3)

In [34]:
len(hist_div_mod), len(hist_mod)

(1, 3)

In [35]:
len(hist_gar_mod), len(hist_mod)

(2, 3)

In [36]:
len(hist_renda_mod), len(hist_mod)

(1, 3)

In [37]:
pm = (len(hist_mod) / len(df)) \
* (len(hist_boa_mod) / len(hist_mod)) \
* (len(hist_div_mod) / len(hist_mod)) \
* (len(hist_gar_mod) / len(hist_mod)) \
* (len(hist_renda_mod) / len(hist_mod)) 
pm

0.005291005291005291

In [38]:
hist_baixo = df.query('Risco == "Baixo"')
hist_baixo

,Historico,Divida,Garantias,Renda,Risco
4,Desconhecida,Baixa,Nenhuma,> 35.000,Baixo
5,Desconhecida,Baixa,Adequada,> 35.000,Baixo
8,Boa,Baixa,Nenhuma,> 35.000,Baixo
9,Boa,Alta,Adequada,> 35.000,Baixo
12,Boa,Alta,Nenhuma,> 35.000,Baixo


In [39]:
hist_boa_baixo = df.query('Historico == "Boa" & Risco == "Baixo"')
hist_boa_baixo

,Historico,Divida,Garantias,Renda,Risco
8,Boa,Baixa,Nenhuma,> 35.000,Baixo
9,Boa,Alta,Adequada,> 35.000,Baixo
12,Boa,Alta,Nenhuma,> 35.000,Baixo


In [40]:
hist_div_baixo = df.query('Risco == "Baixo" & Divida == "Alta"')
hist_div_baixo

,Historico,Divida,Garantias,Renda,Risco
9,Boa,Alta,Adequada,> 35.000,Baixo
12,Boa,Alta,Nenhuma,> 35.000,Baixo


In [41]:
hist_gar_baixo = df.query('Garantias == "Nenhuma" & Risco == "Baixo"')
hist_gar_baixo

,Historico,Divida,Garantias,Renda,Risco
4,Desconhecida,Baixa,Nenhuma,> 35.000,Baixo
8,Boa,Baixa,Nenhuma,> 35.000,Baixo
12,Boa,Alta,Nenhuma,> 35.000,Baixo


In [42]:
hist_renda_baixo = df.query('Renda == "> 35.000" & Risco == "Baixo"')
hist_renda_baixo

,Historico,Divida,Garantias,Renda,Risco
4,Desconhecida,Baixa,Nenhuma,> 35.000,Baixo
5,Desconhecida,Baixa,Adequada,> 35.000,Baixo
8,Boa,Baixa,Nenhuma,> 35.000,Baixo
9,Boa,Alta,Adequada,> 35.000,Baixo
12,Boa,Alta,Nenhuma,> 35.000,Baixo


In [43]:
pb = (len(hist_baixo) / len(df)) \
* (len(hist_boa_baixo) / len(hist_baixo)) \
* (len(hist_div_baixo) / len(hist_baixo)) \
* (len(hist_gar_baixo) / len(hist_baixo)) \
* ((len(hist_renda_baixo)) / len(hist_baixo)) 
pb

0.05142857142857143

In [44]:
pa, pm, pb

(0.007936507936507936, 0.005291005291005291, 0.05142857142857143)

In [45]:
sum_ps = pa + pm + pb
sum_ps

0.06465608465608466

In [46]:
pa = (pa / sum_ps) * 100
print(f'Probabilidade do risco ser Alto: {round(pa, 2)}%')

Probabilidade do risco ser Alto: 12.27%


In [47]:
pm = (pm / sum_ps) * 100
print(f'Probabilidade do risco ser Moderado: {round(pm, 2)}%')

Probabilidade do risco ser Moderado: 8.18%


In [48]:
pb = (pb / sum_ps) * 100
print(f'Probabilidade do risco ser baixo: {round(pb, 2)}%')

Probabilidade do risco ser baixo: 79.54%


In [49]:
12.27 + 8.18 + 79.54

99.99000000000001

In [50]:
target = "Risco"
first = df.drop(target, axis=1).columns
first

Index(['Historico', 'Divida', 'Garantias', 'Renda'], dtype='object')

In [51]:
micolumns = []
for col in first:
    for val in df[col].unique():
        micolumns.append((col, val))
        
micolumns.append(('Total','Proba'))

In [52]:
df_mult = pd.MultiIndex.from_tuples(micolumns, names=['Classe',target])
print(df_mult)

MultiIndex([('Historico',               'Ruim'),
            ('Historico',       'Desconhecida'),
            ('Historico',                'Boa'),
            (   'Divida',               'Alta'),
            (   'Divida',              'Baixa'),
            ('Garantias',            'Nenhuma'),
            ('Garantias',           'Adequada'),
            (    'Renda',           '< 15.000'),
            (    'Renda', '>= 15.000 a 35.000'),
            (    'Renda',           '> 35.000'),
            (    'Total',              'Proba')],
           names=['Classe', 'Risco'])


In [53]:
dfmi = pd.DataFrame(columns = df_mult, index=sorted(df[target].unique()))
dfmi

Classe   Historico                   Divida       Garantias             Renda  \
Risco         Ruim Desconhecida  Boa   Alta Baixa   Nenhuma Adequada < 15.000   
Alto           NaN          NaN  NaN    NaN   NaN       NaN      NaN      NaN   
Baixo          NaN          NaN  NaN    NaN   NaN       NaN      NaN      NaN   
Moderado       NaN          NaN  NaN    NaN   NaN       NaN      NaN      NaN   

Classe                               Total  
Risco    >= 15.000 a 35.000 > 35.000 Proba  
Alto                    NaN      NaN   NaN  
Baixo                   NaN      NaN   NaN  
Moderado                NaN      NaN   NaN

In [54]:
dfmi.loc[tuple(sorted(df[target].unique())),("Total")] = (df[target].value_counts() / len(df)).values

In [55]:
dfmi

Classe   Historico                   Divida       Garantias             Renda  \
Risco         Ruim Desconhecida  Boa   Alta Baixa   Nenhuma Adequada < 15.000   
Alto           NaN          NaN  NaN    NaN   NaN       NaN      NaN      NaN   
Baixo          NaN          NaN  NaN    NaN   NaN       NaN      NaN      NaN   
Moderado       NaN          NaN  NaN    NaN   NaN       NaN      NaN      NaN   

Classe                                   Total  
Risco    >= 15.000 a 35.000 > 35.000     Proba  
Alto                    NaN      NaN  0.428571  
Baixo                   NaN      NaN  0.357143  
Moderado                NaN      NaN  0.214286

In [56]:
for classe, col in dfmi.columns[:-1]:
    values = []
    for idx in dfmi.index:
        freq = len(df.query(f'{target} == "{idx}" & {classe} == "{col}"'))
        freq_cls = len(df.query(f'{target} == "{idx}"'))
        if freq == 0:
            freq = 1
            
        values.append(freq / freq_cls)
            
    dfmi.loc[tuple(dfmi.index),(classe, col)] = values
        
dfmi 

Classe   Historico                           Divida           Garantias  \
Risco         Ruim Desconhecida       Boa      Alta     Baixa   Nenhuma   
Alto      0.500000     0.333333  0.166667  0.666667  0.333333  1.000000   
Baixo     0.200000     0.400000  0.600000  0.400000  0.600000  0.600000   
Moderado  0.333333     0.333333  0.333333  0.333333  0.666667  0.666667   

Classe                 Renda                                  Total  
Risco     Adequada  < 15.000 >= 15.000 a 35.000  > 35.000     Proba  
Alto      0.166667  0.666667           0.333333  0.166667  0.428571  
Baixo     0.400000  0.200000           0.200000  1.000000  0.357143  
Moderado  0.333333  0.333333           0.666667  0.333333  0.214286

In [57]:
x

,Historico,Divida,Garantias,Renda,Risco
12,Boa,Alta,Nenhuma,> 35.000,Baixo


In [58]:
tuple(x.columns[:-1])

('Historico', 'Divida', 'Garantias', 'Renda')

In [59]:
tuple(x.values[0][:-1])

('Boa', 'Alta', 'Nenhuma', '> 35.000')

In [60]:
x = [col for col in zip(tuple(x.columns[:-1]), tuple(x.values[0][:-1]))]
x

[('Historico', 'Boa'),
 ('Divida', 'Alta'),
 ('Garantias', 'Nenhuma'),
 ('Renda', '> 35.000')]

In [61]:
dfmi[('Total','Proba')]

Alto        0.428571
Baixo       0.357143
Moderado    0.214286
Name: (Total, Proba), dtype: float64

In [62]:
df_x = dfmi.loc[tuple(dfmi.index), (x)]
df_x[('Total','Proba')] = dfmi[('Total','Proba')]
df_x

Classe,Historico,Divida,Garantias,Renda,Total
Risco,Boa,Alta,Nenhuma,> 35.000,Proba
Alto,0.166667,0.666667,1.000000,0.166667,0.428571
Baixo,0.600000,0.400000,0.600000,1.000000,0.357143
Moderado,0.333333,0.333333,0.666667,0.333333,0.214286


In [63]:
sum_probs = []
for line in df_x.iterrows():
    p = 1
    for val in line[1].values:
        p*=val
    sum_probs.append(p)
    
sum_probs

[0.007936507936507936, 0.05142857142857143, 0.005291005291005291]

In [64]:
summatory = sum(sum_probs)
summatory

0.06465608465608466

In [65]:
df_x.index[0]

'Alto'

In [66]:
cls_prob = {}
for i, val in enumerate(sum_probs):
    cls_prob[df_x.index[i]] = f'{(val / summatory) * 100}%'
    
cls_prob

{'Alto': '12.27495908346972%',
 'Baixo': '79.54173486088378%',
 'Moderado': '8.183306055646481%'}

In [67]:
class NaiveBayes:
    
    
    def __init__(self, df:pd.DataFrame, target:str):
        self.data = df
        self.target = target
        self.df = self.__build_df()
        self.proba = None
        
        
    def __format_columns(self, cols:list) -> pd.MultiIndex:
        
        micolumns = []
        for col in first:
            for val in df[col].unique():
                micolumns.append((col, val))

        micolumns.append(('Total','Proba'))
        
        return pd.MultiIndex.from_tuples(micolumns, names=['Classe',target])
    
    
    def __prob_calculates(self) -> pd.DataFrame:
        
       
        for classe, col in self.df.columns[:-1]:
            values = []
            for idx in self.df.index:
                freq = len(self.data.query(f'{target} == "{idx}" & {classe} == "{col}"'))
                freq_cls = len(self.data.query(f'{target} == "{idx}"'))
                if freq == 0:
                    freq = 1

                values.append(freq / freq_cls)

            self.df.loc[tuple(self.df.index),(classe, col)] = values
            
        self.df.loc[tuple(sorted(self.data[target].unique())),("Total")] = (
            self.data[target].value_counts() / len(self.data)).values
            
        return self.df
        
    
    def __build_df(self) -> pd.DataFrame:
        
        cols = self.data.drop(self.target, axis=1).columns
        
        df_mult = self.__format_columns(cols)
        self.df = pd.DataFrame(columns = df_mult, index=sorted(df[target].unique()))
        
        return self.__prob_calculates()
    
    def __sum_calculates(self, x):
        sum_probs = []
        for line in df_x.iterrows():
            p = 1
            for val in line[1].values:
                p*=val
            sum_probs.append(p)
        
        return sum(sum_probs)
    
        
    def get_nb(self):
        return self.df
    
    def classify(self, x: list) -> dict:
        cols = self.data.drop(self.target, axis=1).columns
        x = [col for col in zip(cols, x)]
        
        df_x = dfmi.loc[tuple(dfmi.index), (x)]
        df_x[('Total','Proba')] = dfmi[('Total','Proba')]
        
        cls_prob = {}
        for i, val in enumerate(sum_probs):
            cls_prob[df_x.index[i]] = f'{(val / summatory) * 100}%'
            
        return cls_prob
        

In [68]:
from Models.NaiveBayes import NaiveBayes

In [69]:
nb = NaiveBayes(df, 'Risco')
nb.get_nb()

Classe   Historico                           Divida           Garantias  \
Risco         Ruim Desconhecida       Boa      Alta     Baixa   Nenhuma   
Alto           0.5     0.333333  0.166667  0.666667  0.333333         1   
Baixo          0.2          0.4       0.6       0.4       0.6       0.6   
Moderado  0.333333     0.333333  0.333333  0.333333  0.666667  0.666667   

Classe                 Renda                                  Total  
Risco     Adequada  < 15.000 >= 15.000 a 35.000  > 35.000     Proba  
Alto      0.166667  0.666667           0.333333  0.166667  0.428571  
Baixo          0.4       0.2                0.2         1  0.357143  
Moderado  0.333333  0.333333           0.666667  0.333333  0.214286

In [70]:
x = ['Boa', 'Alta', 'Nenhuma', '> 35.000']

nb.classify(x)

{'Alto': '12.27%', 'Baixo': '79.54%', 'Moderado': '8.18%'}

In [71]:
x = ['Ruim', 'Alta', 'Nenhuma', '> 35.000']
nb.classify(x)

{'Alto': '51.49%', 'Baixo': '37.07%', 'Moderado': '11.44%'}

In [72]:
x = ['Boa', 'Baixa', 'Adequada', '> 35.000']
nb.classify(x)

{'Alto': '1.15%', 'Baixo': '89.63%', 'Moderado': '9.22%'}

In [82]:
x = ['Ruim', 'Baixa', 'Adequada', '>= 15.000 a 35.000']
nb.classify(x)

{'Alto': '22.07%', 'Baixo': '19.07%', 'Moderado': '58.86%'}